In [14]:
import pandas as pd
import os

# Path to the directory containing your CSV files
folder_path = "../yfinance_data/the file"  # Make sure this path is correct

# Dictionary to hold individual DataFrames
data_dict = {}

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        stock_name = filename.replace(".csv", "")
        file_path = os.path.join(folder_path, filename)
        
        # Load CSV into DataFrame with datetime index
        df = pd.read_csv(file_path, parse_dates=True, index_col=0)
        
        # Ensure index is datetime and add normalized date
        df.index = pd.to_datetime(df.index)
        df['Date'] = df.index.normalize()
        
        # Save to dictionary
        data_dict[stock_name] = df
   # Create a new column with just the date

In [20]:
# For news data
news_df=pd.read_csv("C:/Users/Student/Documents/kaim documents/week 1/raw_analyst_ratings.csv/raw_analyst_ratings.csv")
# Step 1: Convert to datetime (safely)
news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce', infer_datetime_format=True)
print(news_df['date'].dtype)
print(news_df['date'].head())


C:\Users\Student\AppData\Local\Temp\ipykernel_18028\4162999943.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce', infer_datetime_format=True)


datetime64[ns, UTC-04:00]
0   2020-06-05 10:30:54-04:00
1   2020-06-03 10:45:20-04:00
2   2020-05-26 04:30:07-04:00
3   2020-05-22 12:45:06-04:00
4   2020-05-22 11:38:59-04:00
Name: date, dtype: datetime64[ns, UTC-04:00]


In [25]:
news_df = pd.read_csv("C:/Users/Student/Documents/kaim documents/week 1/raw_analyst_ratings.csv/raw_analyst_ratings.csv")

news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce', infer_datetime_format=True)

print(news_df['date'].dtype)
print(news_df['date'].head())

news_df = news_df.dropna(subset=['date'])  # Drop NaT values

# Remove timezone to avoid errors with .dt.normalize()
news_df['date'] = news_df['date'].dt.tz_localize(None)

# Normalize date to midnight
news_df['date'] = news_df['date'].dt.normalize()
news_df = news_df.dropna(subset=['date'])

print(news_df['date'].head())
print(news_df['date'].dtype)


C:\Users\Student\AppData\Local\Temp\ipykernel_18028\2068501246.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce', infer_datetime_format=True)


datetime64[ns, UTC-04:00]
0   2020-06-05 10:30:54-04:00
1   2020-06-03 10:45:20-04:00
2   2020-05-26 04:30:07-04:00
3   2020-05-22 12:45:06-04:00
4   2020-05-22 11:38:59-04:00
Name: date, dtype: datetime64[ns, UTC-04:00]
0   2020-06-05
1   2020-06-03
2   2020-05-26
3   2020-05-22
4   2020-05-22
Name: date, dtype: datetime64[ns]
datetime64[ns]


In [26]:
import pandas as pd
import numpy as np
import os
from datetime import time
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from textblob import TextBlob

In [35]:
# 1. Load your datasets
def load_data():
    """Load news and stock data from your specific paths"""
    # Load news data
    news_df = pd.read_csv("C:/Users/Student/Documents/kaim documents/week 1/raw_analyst_ratings.csv/raw_analyst_ratings.csv")
    
    # Load stock data from multiple files
    folder_path = "../yfinance_data/the file"
    data_dict = {}
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            stock_name = filename.replace(".csv", "")
            file_path = os.path.join(folder_path, filename)
            data_dict[stock_name] = pd.read_csv(file_path, parse_dates=True, index_col=0)
    
    return news_df, data_dict
news_df, data_dict = load_data()


In [53]:
# 2. Preprocess and align data
def preprocess_data(news_df, data_dict):
    """Clean and prepare data for analysis"""
    # Convert news timestamp to datetime
    news_df = news_df.copy()
    news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce')
    news_df =news_df.dropna(subset=['date'])  # Remove rows where conversion failed
    news_df['date'] =news_df['date'].dt.normalize().dt.tz_localize(None)
    

    
    # Create a combined stock DataFrame with ticker information
    stock_dfs = []
    for ticker, df in data_dict.items():
        temp_df = df.copy()
        temp_df['Ticker'] = ticker
        stock_dfs.append(temp_df)
    
    combined_stocks=pd.concat(stock_dfs)
    combined_stocks.index = pd.to_datetime(combined_stocks.index)

    return news_df, combined_stocks
# Call the function and store the result

# Now preprocess and align the data
clean_news_df, combined_stocks_df = preprocess_data(news_df, data_dict)

# Check the results
print(combined_stocks_df.head())
print(combined_stocks_df.columns)


                Open      High       Low     Close  Adj Close     Volume  \
Date                                                                       
1980-12-12  0.128348  0.128906  0.128348  0.128348   0.098943  469033600   
1980-12-15  0.122210  0.122210  0.121652  0.121652   0.093781  175884800   
1980-12-16  0.113281  0.113281  0.112723  0.112723   0.086898  105728000   
1980-12-17  0.115513  0.116071  0.115513  0.115513   0.089049   86441600   
1980-12-18  0.118862  0.119420  0.118862  0.118862   0.091630   73449600   

            Dividends  Stock Splits                Ticker  
Date                                                       
1980-12-12        0.0           0.0  AAPL_historical_data  
1980-12-15        0.0           0.0  AAPL_historical_data  
1980-12-16        0.0           0.0  AAPL_historical_data  
1980-12-17        0.0           0.0  AAPL_historical_data  
1980-12-18        0.0           0.0  AAPL_historical_data  
Index(['Open', 'High', 'Low', 'Close', 'Adj Clo

In [43]:
# Parse datetime with timezone info
news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce')

# Convert to timezone naive (optional)
news_df['date'] = news_df['date'].dt.tz_localize(None)

# Normalize to remove time (keep date only)
news_df['date'] = news_df['date'].dt.normalize()

print(news_df['date'].dtype)
print(news_df['date'].head())

datetime64[ns]
0   2020-06-05
1   2020-06-03
2   2020-05-26
3   2020-05-22
4   2020-05-22
Name: date, dtype: datetime64[ns]


In [47]:
from datetime import time
import pandas as pd

def align_news_with_stocks(news_df, combined_stocks_df):
    """Assign each news article to a valid trading day in the stock data."""
    
    # Define market open and close times
    MARKET_OPEN = time(9, 30)
    MARKET_CLOSE = time(16, 0)

    # Get unique trading days (dates only) from stock_dfs index
    trading_days = combined_stocks_df.index.normalize().unique()

    def assign_trading_day(date):
        if pd.isnull(date):
            return None

        ts_time = date.time()
        ts_date = date.date()
        day = pd.Timestamp(ts_date)

        # During market hours: assign to that day if valid
        if MARKET_OPEN <= ts_time <= MARKET_CLOSE:
            return day if day in trading_days else None

        # After market close: assign to next trading day
        elif ts_time > MARKET_CLOSE:
            next_day = day + pd.Timedelta(days=1)
            while next_day not in trading_days:
                next_day += pd.Timedelta(days=1)
            return next_day

        # Before market open: assign to same day if valid
        else:
            return day if day in trading_days else None

    # Apply the function to assign trading day for each news date
    news_df['trading_day'] = news_df['date'].apply(assign_trading_day)

    # Drop rows where trading_day assignment failed
    aligned_news = news_df.dropna(subset=['trading_day'])

    return aligned_news

# Example usage:
# Make sure news_df and stock_df are defined and have proper datetime indices
# For example:
# news_df['date'] should be datetime64[ns]
# stock_df.index should be datetime64[ns]

aligned_news = align_news_with_stocks(news_df, combined_stocks_df)

# Print info to verify
print(aligned_news.columns)
# aligned_news.info()


Index(['Unnamed: 0', 'headline', 'url', 'publisher', 'date', 'stock',
       'trading_day'],
      dtype='object')


In [48]:
print("News shape:", news_df.shape)
print("Stock shape:", combined_stocks_df.shape)
print("Aligned News shape:", aligned_news.shape)

print("Missing in aligned_news:")
print(aligned_news.isnull().sum())


News shape: (1407328, 7)
Stock shape: (45428, 9)
Aligned News shape: (55230, 7)
Missing in aligned_news:
Unnamed: 0     0
headline       0
url            0
publisher      0
date           0
stock          0
trading_day    0
dtype: int64


In [52]:
print(news_df.columns)
print(combined_stocks_df.columns)
print(aligned_news.columns)


Index(['Unnamed: 0', 'headline', 'url', 'publisher', 'date', 'stock',
       'trading_day'],
      dtype='object')
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Dividends',
       'Stock Splits', 'Ticker'],
      dtype='object')
Index(['Unnamed: 0', 'headline', 'url', 'publisher', 'date', 'stock',
       'trading_day'],
      dtype='object')


In [54]:
combined_stocks_df = combined_stocks_df.reset_index()
print(combined_stocks_df.columns)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Dividends', 'Stock Splits', 'Ticker'],
      dtype='object')


In [56]:
# Assuming 'trading_day' column in aligned news and stock_df['date'] exists
aligned_news['trading_day'] = pd.to_datetime(aligned_news['trading_day'])
combined_stocks_df['Date'] = pd.to_datetime(combined_stocks_df['Date'])  # or from index if needed

# Merge on 'trading_day'
merged_df = pd.merge(aligned_news, combined_stocks_df, left_on='trading_day', right_on='Date', how='inner')
print(merged_df.columns)

Index(['Unnamed: 0', 'headline', 'url', 'publisher', 'date', 'stock',
       'trading_day', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close',
       'Volume', 'Dividends', 'Stock Splits', 'Ticker'],
      dtype='object')


C:\Users\Student\AppData\Local\Temp\ipykernel_18028\3893708251.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aligned_news['trading_day'] = pd.to_datetime(aligned_news['trading_day'])
